In [1]:
!pip install -q git+https://github.com/huggingface/transformers

In [2]:
model_path = '/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1'

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_path)

In [4]:
from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained(model_path,
                                            torch_dtype=torch.bfloat16,
                                            device_map='auto',
                                            trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
messages = [{"role":"user",
            "content":"Can you tell us 3 cities to visit in Turkey"}]

In [6]:
tokenizer.apply_chat_template(messages, tokenize=False)

'<s>[INST] Can you tell us 3 cities to visit in Turkey [/INST]'

In [7]:
model_inputs = tokenizer.apply_chat_template(messages,
                                             return_tensors='pt')
print(model_inputs)

tensor([[    1,   733, 16289, 28793,  2418,   368,  1912,   592, 28705, 28770,
          9245,   298,  3251,   297, 17915,   733, 28748, 16289, 28793]])


In [9]:
torch.backends.cuda.enable_mem_efficient_sdp(False)

torch.backends.cuda.enable_flash_sdp(False)

In [10]:
generated_ids = model.generate(
    model_inputs,
    max_new_tokens=1000,
    do_sample=True,
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
2024-03-14 21:32:23.350616: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-14 21:32:23.350736: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-14 21:32:23.522206: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [11]:
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

<s> [INST] Can you tell us 3 cities to visit in Turkey [/INST] Certainly! Turkey is a beautiful country with many interesting cities to explore. Here are three cities that are definitely worth a visit:
1. Istanbul - This is the largest and most populous city in Turkey, and it's also the Cultural Capital of Europe. Istanbul is located on both sides of the Bosphorus Strait, which separates Asia from Europe. Some of the top sites to see in Istanbul include the Hagia Sophia, the Blue Mosque, and Topkapi Palace.
2. Alaçati - This is a charming coastal town on the Aegean Sea, and it's known for its beautiful beaches, crystal-clear waters, and historic architecture. Alaçati is a great place to relax and unwind, or you can explore the local shops, cafes, and restaurants.
3. Cappadocia - This is a fascinating region located in southeastern Turkey, and it's famous for its unique and otherworldly landscapes. Cappadocia is home to rugged mountains, underground caves, and surreal rock formations, a

In [12]:
messages = [{
    "role":"user",
    "content":"Act as a goutmet chef. I have a friend. I want to imress my friend with a special vegetarian dish. \
    What do you recommend? \
    Give me two options, along with the whole recipe for each"
}]

model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt")

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)

decode = tokenizer.batch_decode(generated_ids)
print(decode[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1460: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


<s> [INST] Act as a goutmet chef. I have a friend. I want to imress my friend with a special vegetarian dish.     What do you recommend?     Give me two options, along with the whole recipe for each [/INST] As a gourmet chef, it's always a delight to cook for friends who appreciate and savor good food. Here are two vegetarian dishes that I recommend:

Option 1: Vegetable Paella

Ingredients:

* 1/2 cup olive oil
* 1 onion, finely chopped
* 4 cloves garlic, minced
* 1 red pepper, diced
* 1 green pepper, diced
* 1 pound boneless, skinless chicken breasts, cut into smaller pieces
* 1 cup arborio rice
* 1/4 cup white wine
* 2 cups vegetable broth
* 1 teaspoon saffron threads
* 1 teaspoon paprika
* Salt and pepper to taste
* chopped fresh parsley for garnish

Instructions:

1. In a large, shallow pan, heat the olive oil over medium heat. Add the onions, garlic, and peppers, and cook until they are soft and lightly browned.
2. Add the chicken to the pan, and cook until it is browned on all s

In [16]:
messages = [{
    "role":"user",
    "content":"How many helicopters can a human eat in one sitting?"
}]

model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt")

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)

decode = tokenizer.batch_decode(generated_ids)
print(decode[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] How many helicopters can a human eat in one sitting? [/INST] It is not possible for a human to eat a helicopter in one sitting, as helicopters are not designed to be consumed by humans. They are aircraft designed for transportation or other purposes, not for consumption.</s>


In [17]:
messages = [
    {
        "role":"user",
        "content":"What fascinates you about AI?"
    },
    {
        "role":"assistant",
        "content":"I'm fascinated by AI's data analysis and prediction abilities. \
        It has the potential to revolutionize industries and improve problem-solving"
    },
    {
        "role":"user",
        "content":"Should people be afraid of AI?"
    }
]

model_inputs = tokenizer.apply_chat_template(messages, return_tensors='pt')

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)

decode = tokenizer.batch_decode(generated_ids)
print(decode[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1460: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


<s> [INST] What fascinates you about AI? [/INST]I'm fascinated by AI's data analysis and prediction abilities.         It has the potential to revolutionize industries and improve problem-solving</s>  [INST] Should people be afraid of AI? [/INST] There is certainly the possibility that AI could be used for harmful purposes or cause job losses, but we should not just fear it. Instead, we should focus on regulating & developing AI responsibly so it benefits society as a whole.</s>


In [18]:
messages = [{
    "role":"user",
    "content":"Türkiye'de en fazla ziyaret edilen 3 şehir hangisidir?",
}]

model_inputs = tokenizer.apply_chat_template(messages, return_tensors='pt')

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)

decode = tokenizer.batch_decode(generated_ids)
print(decode[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1460: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


<s> [INST] Türkiye'de en fazla ziyaret edilen 3 şehir hangisidir? [/INST] Antalya, İstanbul ve Ankara</s>
